In [1]:
import csv
import pandas as pd
import numpy as np
import plotly.express as px
import cpi

In [2]:
df = pd.read_csv("./unprocessed_dataset.zip")

C:\Users\berpr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (13,15,16,18,32,34,36,37,41,45,46,55,71,83,84,86,87,90,91,92,96,98,100,101,103,104,105,106,107,108,109) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
ml_vars = df[['YYYY', 'YYYYMM', 'INCOME', 'HOMEAMT', 'HOMEOWN', 'INVAMT', 'AGE', 'REGION', 'SEX', 'MARRY', 'EGRADE', 'EHSGRD', 'ECLGRD', 'POLAFF']]

test = ml_vars.replace(r'^\s*$', np.nan, regex=True)

test = test.dropna()

test = test.apply(pd.to_numeric, axis=1)

In [4]:
test.to_csv("dataset_to_clean.zip", index= False, compression = "zip")

In [5]:
#Notes

#drop 9999998, 9999999 from homeamt, invamt? 
#drop 99 from homeown
#drop 98, 99 from egrade
# drop 8,9 from ECLGRD, ehsgrd
# drop 8,9 polaff
#
#can drop DKs and NAs from data


In [6]:
# home_vals = [9999999, 9999998]
# inv_vals = [99999998]

# test = test[test.INVAMT.isin(inv_vals) == False]
# test = test[test.HOMEAMT.isin(home_vals) == False]

In [7]:
testing = pd.read_csv("./dataset_to_clean.zip")
testing

,YYYY,YYYYMM,INCOME,HOMEAMT,HOMEOWN,INVAMT,AGE,REGION,SEX,MARRY,EGRADE,EHSGRD,ECLGRD,POLAFF
0,2006,200609,52000,9999998,1,130000,52,1,1,5,16,1,1,2
1,2006,200609,150000,9999998,1,75000,70,3,2,1,17,1,1,3
2,2006,200609,38000,9999998,1,99999998,60,4,2,1,15,1,5,3
3,2006,200609,175000,9999998,1,750000,53,1,1,1,17,1,1,1
4,2006,200609,24000,9999998,2,5001,59,4,1,3,14,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57295,2023,202302,85000,9999998,2,62500,24,1,1,5,16,1,1,2
57296,2023,202302,250000,900000,1,350000,34,4,1,1,17,1,1,2
57297,2023,202302,50000,9999998,1,99999998,59,2,1,1,14,1,1,3
57298,2023,202302,55000,9999998,2,40000,56,4,2,1,16,1,1,3


In [8]:
df = testing
income_columns = ["INCOME","HOMEAMT","HOMEOWN","INVAMT"]
mean_homeamt = df[df["HOMEAMT"] < 9999998].mean()
df = df.replace({"HOMEAMT" : {9999998: np.nan, 9999999: np.nan}, "INVAMT": {99999998: np.nan}})

ref_year = 2020
# can't do inflation with year 2023
df = df[df["YYYY"] != 2023]
df["INFL_HOMEAMT"] = df.apply(lambda x: cpi.inflate(x["HOMEAMT"], int(x["YYYY"]), 2020), axis=1)
df["INFL_INVAMT"] = df.apply(lambda x: cpi.inflate(x["INVAMT"], int(x["YYYY"]), 2020), axis=1)

In [9]:
df = df.drop(["HOMEAMT", "INVAMT"], axis=1)

In [10]:
home_mean = df[df["INFL_HOMEAMT"].notnull()]["INFL_HOMEAMT"].mean()
inv_mean = df[df["INFL_INVAMT"].notnull()]["INFL_INVAMT"].mean()
df = df.replace({"INFL_HOMEAMT": {np.nan: home_mean}, "INFL_INVAMT": {np.nan: inv_mean}})
df.head()

,YYYY,YYYYMM,INCOME,HOMEOWN,AGE,REGION,SEX,MARRY,EGRADE,EHSGRD,ECLGRD,POLAFF,INFL_HOMEAMT,INFL_INVAMT
0,2006,200609,52000,1,52,1,1,5,16,1,1,2,386127.458739,166892.013889
1,2006,200609,150000,1,70,3,2,1,17,1,1,3,386127.458739,96283.854167
2,2006,200609,38000,1,60,4,2,1,15,1,5,3,386127.458739,437017.193297
3,2006,200609,175000,1,53,1,1,1,17,1,1,1,386127.458739,962838.541667
4,2006,200609,24000,2,59,4,1,3,14,1,1,1,386127.458739,6420.207396


In [11]:
# remove unavailable homeownership data
df = df[df["HOMEOWN"] != 99]